# PLANTILLA PARA MÓDULOS DE PREPROCESAMIENTO

In [ ]:
# ---------------------------------------------
# ATENCION - FIJAR ESTAS VARIABLES ANTES DE EJECUTAR
# Este script configura el entorno de trabajo para el proyecto TFM_EVA_MARTIN.
# Asegúrate de que las variables 'nombre_lote' y 'nombre_modulo' estén correctamente definidas
# antes de ejecutar el script.
# ---------------------------------------------

nombre_lote = "LOTE_AAAAMMDD"

nombre_modulo = "MODULO_XX"

# ---------------------------------------------
# Configuración del entorno (Colab y Local)
# Todo el código que se ejecute en Colab debe estar en el directorio de Google Drive
# y debe ser accesible desde el entorno local.
# Si se ejecuta en Colab, se montará Google Drive.
# Si se ejecuta localmente, se utilizará la ruta especificada.
# ---------------------------------------------

try:
    import google.colab
    EN_COLAB = True
except ImportError:
    EN_COLAB = False

if EN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    ruta_base = "/content/drive/MyDrive/TFM_EVA_MARTIN/Modulos"
else:
    ruta_base = "G:/Mi unidad/TFM_EVA_MARTIN/Modulos"

print(f"Entorno detectado: {'Google Colab' if EN_COLAB else 'Local'}")
print(f"Ruta base: {ruta_base}")

lote_id = nombre_lote.replace("LOTE_", "")

Entorno detectado: Local
Ruta base: G:/Mi unidad/TFM_EVA_MARTIN/Modulos


In [ ]:
# ------------------------------------
# Cargar utilidades comunes y configuración
# Utilidades comunes y configuración del proyecto.  
# Estas utilidades se utilizan en todos los módulos del proyecto.
# Se cargan desde el directorio base del proyecto.
# ------------------------------------
import os
import sys
import pandas as pd
ruta_config = os.path.join(ruta_base, "config.yaml")

if ruta_base not in sys.path:
    sys.path.append(ruta_base)
import yaml
import utilidades_comunes

In [ ]:
# ------------------------------------
# PASO 1: Configurar logger                 
# Configura el logger para registrar eventos y errores.
# El logger se guardará en la ruta especificada dentro del directorio del módulo.
# El nombre del logger se construye utilizando el nombre del módulo y el ID del lote.
# El logger se configurará para registrar mensajes en un archivo de log y en la consola.
# El archivo de log se guardará en la ruta de logs especificada en el entorno.
# El nombre del archivo de log se construye utilizando el nombre del módulo y el ID del lote.
# El logger se configurará para registrar mensajes de nivel DEBUG y superiores. 
# ------------------------------------
# 1. Configurar logger
logger = utilidades_comunes.configurar_logger(nombre_modulo, ruta_logs=os.path.join(ruta_base, nombre_modulo, "logs"))

# 2. Inicializar entorno
# Inicializa el entorno de trabajo para el módulo.
# Crea un objeto de entorno que contiene la configuración del módulo, las rutas de entrada y salida, y el logger.
# El entorno se inicializa utilizando las utilidades comunes y la configuración del proyecto.
# El entorno se utilizará para cargar y procesar los datos, así como para guardar los resultados.
# El entorno se inicializa con el nombre del módulo, el ID del lote, la ruta base y la ruta de configuración.
entorno = utilidades_comunes.inicializar_entorno(nombre_modulo, nombre_lote, ruta_base, ruta_config, logger=logger)


In [ ]:
# ------------------------------------
# PASO 2: Cargar y analizar dataset de entrada  
# Carga el dataset de entrada utilizando las utilidades comunes.
# El dataset se busca en la ruta de entrada especificada en el entorno.
# Se utiliza un patrón de búsqueda para encontrar el archivo correspondiente al lote.
# Si no se encuentra el archivo, se lanza una excepción.
# El dataset se carga en un DataFrame de pandas y se muestra una muestra del mismo.
# También se guarda una muestra del dataset en la ruta de ejemplos especificada en el entorno.
# El nombre del archivo de entrada se construye utilizando el nombre del módulo anterior y el ID del lote.
# El patrón de búsqueda se construye utilizando la ruta de entrada, el nombre del módulo anterior y el ID del lote.
# ------------------------------------

patron_busqueda = os.path.join(
    entorno["ruta_entrada"],
    f"dataset_{entorno['nombre_modulo_anterior'].lower()}_{entorno['lote_id']}_*.csv"
)

import glob
archivos_encontrados = glob.glob(patron_busqueda)

if not archivos_encontrados:
    raise FileNotFoundError(f"No se encontró archivo de entrada para el lote {nombre_lote} con patrón: {patron_busqueda}")

fichero_entrada = archivos_encontrados[0]
df_entrada = utilidades_comunes.cargar_dataset(fichero_entrada, logger=logger)

utilidades_comunes.mostrar_muestra_dataset(df_entrada, "dataset de entrada", logger=logger)
utilidades_comunes.guardar_muestra_dataset(df_entrada, "entrada", entorno["ruta_ejemplos"], logger=logger, n=5)


[Timer] 'cargar_transcripciones_txt' se completó en 0.12 s.


In [ ]:
# ------------------------------------
# PASO 3: Procesamiento específico del módulo
# Procesamiento específico del módulo.  
    
@utilidades_comunes.medir_tiempo
def procesamiento_especifico(df, logger=None):
    registros_antes = len(df)
    df = df.drop_duplicates()
    registros_despues = len(df)
    logger.info(f"Duplicados eliminados: {registros_antes - registros_despues}")
    return df

df_salida = procesamiento_especifico(df_entrada, logger=logger)


In [ ]:
# ------------------------------------
# PASO 4: Validación post-procesamiento
# ------------------------------------
# Este paso garantiza que el DataFrame resultante cumple las condiciones estructurales.
# Se puede extender con validaciones adicionales específicas del módulo.

utilidades_comunes.validar_integridad(df_salida, logger)


2025-06-09 09:18:01,674 - INFO - Validando integridad del DataFrame...
2025-06-09 09:18:01,678 - INFO - Filas: 6, Columnas: 2
2025-06-09 09:18:01,683 - INFO - Valores nulos por columna:\nid_transcripcion    0
transcripcion       0
dtype: int64


In [ ]:
# ------------------------------------
# PASO 5: Guardar dataset salida con nombre estándar
# Guardar el DataFrame resultante en un archivo CSV.
# El nombre del archivo se construye utilizando el nombre del módulo y el ID del lote.
# El archivo se guarda en la ruta de salida especificada en el entorno.
# ------------------------------------
nombre_salida = os.path.join(
    entorno["ruta_salida"],
    f"dataset_{nombre_modulo.lower()}_{entorno['lote_id']}.csv"
)

utilidades_comunes.guardar_dataset(df_salida, nombre_salida, logger=logger)


2025-06-09 09:18:05,679 - INFO - Dataset de salida guardado en: G:/Mi unidad/TFM_EVA_MARTIN/Modulos\MODULO_01\./salida\dataset_modulo_01_20250609_091805.csv


In [ ]:
# ------------------------------------
# PASO 6: Mostrar muestra final
# ------------------------------------
nombre_muestra = f"{nombre_modulo.lower()}_{entorno['lote_id']}"
utilidades_comunes.mostrar_muestra_dataset(df_salida, nombre_muestra, logger=logger)

logger.info(f"✅ Finalización del procesamiento del módulo {nombre_modulo}")
logger.info(f"📦 Dataset final disponible en: {nombre_salida}")


2025-06-09 09:18:14,723 - INFO - \n--- Muestra de Dataset Salida (primeras 5 filas) ---


2025-06-09 09:18:14,728 - INFO -                                                                    id_transcripcion                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

\n--- Muestra de Dataset Salida (primeras 5 filas) ---
                                    id_transcripcion  \
0  Positivo_[965021513]_MERIDIANO_HOGAR_2023-02-2...   
1  Negativo_SD23-03822_[965021512]_MERIDIANO_900_...   
2  Negativo_SD23-03822_[965021512]_MERIDIANO_900_...   
3  Neutro_[965641606]_MERIDIANO_902_2023-02-28_10...   
4  Neutro_SD23-03516_[965021512]_MERIDIANO_900_GE...   

                                       transcripcion  
0  Bienvenido Meridiano. Por su seguridad, esta l...  
1  de la central de asistencia de seguros meridia...  
2  Hola, buenas tardes. Doña PERSONA, soy  de la ...  
3  ¿Sí? Buenos días, ¿habló con don XXXXXX? Sí, s...  
4  Bienvenido Meridiano. Por su seguridad, esta l...  
\n--- Estadísticas básicas ---
                 count unique  \
id_transcripcion     6      6   
transcripcion        6      6   

                                                                top freq  
id_transcripcion  Positivo_[965021513]_MERIDIANO_HOGAR_2023-02-2...    1

In [ ]:
# ------------------------------------
# PASO 7: Guardar muestra en carpeta /ejemplos
# ------------------------------------
nombre_muestra = f"{nombre_modulo.lower()}_{entorno['lote_id']}"
utilidades_comunes.guardar_muestra_dataset(
    df_salida,
    nombre_muestra,
    entorno["ruta_ejemplos"],
    logger=logger
)


2025-06-09 09:18:25,718 - INFO - Guardada muestra de salida en G:/Mi unidad/TFM_EVA_MARTIN/Modulos\MODULO_01\./ejemplos/muestra_modulo_01.csv
